Understanding the PyTorch parallel scan, with RNNs/Mamba in mind

In [2]:
import torch
import math

In [5]:
X = torch.tensor([1,2,3,4])

torch.cumsum(X, dim = 0)

tensor([ 1,  3,  6, 10])

In [22]:
Y =torch.zeros_like(X)
print(Y)

tensor([0, 0, 0, 0])


In [23]:
cumulative_sum = 0
print(X.size(0))

4


In [24]:
for t in range(X.size(0)):
  cumulative_sum += X[t]
  Y[t] = cumulative_sum
Y

tensor([ 1,  3,  6, 10])

In [43]:
X = torch.tensor([1,2,3,4])
Y = torch.zeros_like(X)
Y

tensor([0, 0, 0, 0])

In [46]:
X = torch.tensor([1,2,3,4])
Y = torch.zeros_like(X)
hung = 0
for i in range(X.size(0)):
  hung += X[i]
  Y[i] = hung
Y

tensor([ 1,  3,  6, 10])

In [39]:
Y = torch.zeros_like(X)

cumulative_sum = 0
for t in range(X.size(0)):
    cumulative_sum += X[t]
    Y[t] = cumulative_sum

Y

tensor([ 1,  3,  6, 10])

In [47]:
X = torch.tensor([1,2,3,4])
Y[0] = X[0]
for i in range(1,X.size(0)):
  Y[i] = X[i] + Y[i-1]
Y

tensor([ 1,  3,  6, 10])

In [70]:
x = torch.tensor([1,2,3,4,5,6,7,8])
l = x.size(0)
l

8

In [71]:
xa = x.view(l//2,2) # (dòng , cột)
xa

tensor([[1, 2],
        [3, 4],
        [5, 6],
        [7, 8]])

In [63]:
xa[:,0]

tensor([1, 3, 5, 7])

In [64]:
xa[:,1]

tensor([ 4, 10, 16, 22])

In [65]:
xa[:,0] + xa[:,1]

tensor([ 5, 13, 21, 29])

In [72]:
xa[:,1] += xa[:,0]
xa[:,1]

tensor([ 3,  7, 11, 15])

In [73]:
xa = xa[:,1]
xa = xa.view(xa.size(0)//2,2)

In [74]:
xa

tensor([[ 3,  7],
        [11, 15]])

In [75]:
xa[:,1] += xa[:,0]
xa[:,1]

tensor([[ 3, 10],
        [11, 26]])

In [76]:
xa[:,1]

tensor([10, 26])

In [77]:
xa = xa[:,1]
xa = xa.view(xa.size(0)//2,2)
xa[:,1] += xa[:,0]
xa[:,1]

tensor([36])

In [78]:
X = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8]) # input array
L = X.size(0)

Xa = X

for k in range(int(math.log2(L))):
    T = 2 * (Xa.size(0) // 2)

    # split into 2 groups of pairs of elements
    Xa = Xa.view(T//2, 2)

    # for each pair, add the first to the second
    Xa[:, 1].add_(Xa[:, 0])

    # change the view for the next iteration
    Xa = Xa[:, 1]

X

tensor([ 1,  3,  3, 10,  5, 11,  7, 36])

In [83]:
X = torch.tensor([5, 1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0]) # input array
L = X.size(0)

Xa = X
num_steps = int(math.log2(L))

for k in range(num_steps):
    T = 2 * (Xa.size(0) // 2)

    # split into 2 groups of pairs of elements
    Xa = Xa.view(T//2, 2)

    # for each pair, add the first to the second
    Xa[:, 1].add_(Xa[:, 0])

    # change the view for the next iteration
    Xa = Xa[:, 1]

# down sweep
for k in range(num_steps-1, -1, -1):
    Xa = X[2**k-1:L:2**k]

    T = 2 * (Xa.size(0) // 2)

    if T < Xa.size(0):
        Xa[:, :, -1].add_(Xa[:, :, -2])

    Xa = Xa[:T].view(T//2, 2)

    Xa[1:, 0].add_(Xa[:-1, 1])

X

tensor([ 5,  6,  8, 11, 15, 20, 26, 33, 41, 41, 41, 41, 41, 41, 41, 41])

Thuật toán chia thành 2 giai đoạn:

Upsweep (reduce phase):

Gom cặp phần tử rồi cộng dồn (kiểu như xây “cây nhị phân” của tổng).

Dùng .view(..., 2) để chia thành từng cặp.

Xa[:, 1].add_(Xa[:, 0]) = cộng giá trị phần tử bên trái vào phần tử bên phải.

Lặp nhiều bước, mỗi lần phạm vi cộng dồn tăng gấp đôi.

Downsweep:

Truyền ngược lại giá trị tích lũy để hoàn tất mảng prefix-sum.

Lúc này ta khôi phục prefix sum cho từng phần tử, không chỉ nút “lá phải” trong cây.